In [1]:
%load_ext tensorboard
%tensorboard --logdir ./result

In [2]:
import math
import random
from typing import List, Optional, Tuple, Union
import torch
import torch.nn.functional as F
import torch.utils.checkpoint
from torch import nn
import os
from torch.utils.data import IterableDataset, Dataset
import json
import numpy as np
from transformers import PreTrainedModel
from transformers.modeling_outputs import CausalLMOutputWithPast
from transformers import PretrainedConfig
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer, DefaultDataCollator, DataCollatorForTokenClassification, AutoConfig
from train_llama2_from_scratch import Config, LLM

/Users/yingyao/miniconda3/envs/transformer-practice/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# eval result
AutoConfig.register("custom_gpt", Config)
AutoModelForCausalLM.register(Config, LLM)
reload_model = AutoModelForCausalLM.from_pretrained('./model')  # './model/sft'
tokenizer = AutoTokenizer.from_pretrained("./tokenizer", use_fast=True)
tokenizer.bos_token = '<|im_start|>' # based on original pretrain data
tokenizer.eos_token = '<|im_end|>'
input_ids = [tokenizer.bos_token_id] + tokenizer.encode("1+1等于几?")
input_data = {'input_ids': torch.tensor(input_ids).unsqueeze(0), "labels":None} # unsqueeze(0) to insert a dim at index 0 for batch
input_data

HFValidationError: Repo id must use alphanumeric chars, '-', '_' or '.'. The name cannot start or end with '-' or '.' and the maximum length is 96: './tokenizer'.

In [6]:
for token in reload_model.generate(inputs=input_data, eos=tokenizer.eos_token_id, max_new_tokens=100, stream=False):
    print(tokenizer.decode(token[0]))

13觉。有�卫体<不一�；<<<，<<<<<<<<<<<<<<<|<|<<|<<|<|�<|<|<|<|<|<|<|<|<|im<|<|<|<|im<|�<|im<|<|im<|�<|im|�<|


In [7]:
reload_model = AutoModelForCausalLM.from_pretrained('./model/sft')  # './model/sft'

In [8]:
for token in reload_model.generate(inputs=input_data, eos=tokenizer.eos_token_id, max_new_tokens=100, stream=False):
    print(tokenizer.decode(token[0]))

RuntimeError: probability tensor contains either `inf`, `nan` or element < 0